In [19]:
import torch
import gradio as gr
import os
from transformers import pipeline

'''
Model picker
    -Token prompt
Check point picker
Stream of chats
Export to story button

-Implement saving checkpoints to json
-Implement loading full checkpoint state
-Implement saving previous
-Implement Export Story
'''
pipe = None
token = ""
# Function to unload the model
def unload_model():
    global pipe
    if 'pipe' in globals():
        del pipe
        torch.cuda.empty_cache()

def load_model(model_name):
    global pipe, token
    unload_model()
    if len(token) > 0:
        pipe = pipeline("text-generation", model=model_name, device='cuda', token=token)
        return
    pipe = pipeline("text-generation", model=model_name, device='cuda')
        
def changeToken(newToken):
    global token
    token = newToken

# Close Gradio interface if open
unload_model()
load_model("Qwen/Qwen2-0.5B-Instruct")
if 'iface' in globals() and iface is not None:
    iface.close()
    

# Function to handle model inference
def llama_inference(prompt):
    global pipe
    messages = [{"role": "user", "content": prompt}]
    response = pipe(messages, max_length=1024)
    print(response)
    return response[0]['generated_text'][1]['content']

print(llama_inference('hello'))

css = """
    textarea, .white-background input {
        background-color: white !important;
        color: black !important;
        -webkit-text-fill-color: black !important;
    }
"""

checkPoints = ['CPT1', 'CPT2', 'CPT3', 'CPT4', 'CPT5', 'CPT6']
history = []

# Create the Gradio interface
with gr.Blocks(css=css) as iface:
    global history
    chatbot = gr.Chatbot(elem_id="chatbot")

    with gr.Row():
        with gr.Column(scale=3):
            model_dropdown = gr.Dropdown(
                choices=["Qwen/Qwen2-0.5B-Instruct", "meta-llama/Meta-Llama-3.1-8B"],
                label="Select Model",
                value="Qwen/Qwen2-0.5B-Instruct",
                interactive=True,
                change=load_model
            )
            hf_token_box = gr.Textbox(placeholder="Enter your Hugging Face token...", label="Hugging Face Token", type="password", elem_classes=["white-background"])
            
        
        with gr.Column(scale=7):
            checkpoint_dropdown = gr.Dropdown(choices=checkPoints, label="CheckPoints", interactive=True)
            prompt_input = gr.Textbox(lines=2, placeholder="Enter your prompt here...", label="Prompt", elem_classes=["white-background"])
            submit_button = gr.Button("Submit")
            button = gr.Button("Export Storyline")

    def update_chatbot(prompt):
        response = llama_inference(prompt)
        history.append((prompt, response))
        return history  # Returns a list of tuples (input, response)

    submit_button.click(update_chatbot, [prompt_input], chatbot)
    hf_token_box.change(changeToken, [hf_token_box], None)

# Launch the Gradio app
iface.launch()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Closing server running on port: 7860


C:\Users\jesse\AppData\Local\Temp\ipykernel_6860\3001120504.py:69: GradioUnusedKwargWarning: You have unused kwarg parameters in Dropdown, please remove them: {'change': <function load_model at 0x000001D90159A660>}
  model_dropdown = gr.Dropdown(


[{'generated_text': [{'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}]}]
Hello! How can I assist you today?
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
